In [5]:
from src.schemas.thesaurus.mads import SchemaMads

from src.schemas.settings import Settings
from rdflib import URIRef, Namespace, Graph, Literal, BNode
from rdflib.namespace import RDF, RDFS
settings = Settings()

In [1]:
from src.schemas.catalog.instance import BfInstance
from datetime import datetime

In [2]:
req = {
    "instanceOf": {
        "value": "https://bibliokeia.com/works/8"
    },
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resource": [
        {
            "type": {
                "value": "Instance",
                "label": "Instância"
            }
        },
        {
            "type": {
                "value": "Print",
                "label": "Impresso"
            }
        }
    ],
    "title": {
        "mainTitle": "Sol de feira"
    },
    "physicalDetails": {
        "carrier": {
            "value": "http://id.loc.gov/vocabulary/carriers/nc",
            "label": ""
        },
        "extent": "212 p.",
        "issuance": {
            "value": "http://id.loc.gov/vocabulary/issuance/mono",
            "label": ""
        },
        "media": {
            "value": "http://id.loc.gov/vocabulary/mediaTypes/n",
            "label": ""
        }
    },
    "provisionActivity": [
        {
            "place": {
                "base": "bk",
                "value": "https://bibliokeia.com/authority/81",
                "label": "Manaus (Brazil)"
            },
            "agent": {
                "base": "bk",
                "value": "https://bibliokeia.com/authority/82",
                "label": "Companhia das Letras"
            },
            "date": "2012"
        }
    ],
    "image": {
        "cover": "https://m.media-amazon.com/images/I/41wBgTiEP4L._SY445_SX342_.jpg"
    }
}

In [3]:
request = BfInstance(**req)
request

BfInstance(resource=[Resource(type=BfElement(value='Instance', metadata=None, label='Instância')), Resource(type=BfElement(value='Print', metadata=None, label='Impresso'))], adminMetadata=AdminMetadata(creationDate=None, changeDate=None, status=BfElement(value='http://id.loc.gov/vocabulary/mstatus/n', metadata=None, label='Novo'), descriptionConventions=BfElement(value='http://id.loc.gov/vocabulary/descriptionConventions/aacr', metadata=None, label='AACr'), identifiedBy=None), title=Title(mainTitle='Sol de feira', subtitle=None), variantTitle=None, physicalDetails=PhysicalDetails(carrier=BfElement(value='http://id.loc.gov/vocabulary/carriers/nc', metadata=None, label=''), extent='212 p.', issuance=BfElement(value='http://id.loc.gov/vocabulary/issuance/mono', metadata=None, label=''), media=BfElement(value='http://id.loc.gov/vocabulary/mediaTypes/n', metadata=None, label='')), provisionActivity=[ProvisionActivity(place=BfElement(value='https://bibliokeia.com/authority/81', metadata=None

In [4]:
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = 'TESTE'

In [5]:
from src.function.catalog.instance.graphInstance import MakeGraphInstance


graph = MakeGraphInstance(request)


In [6]:
print(graph)

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://bibliokeia.com/instances/TESTE> a <Instance>,
        <Print> ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            bf:creationDate "2024-06-10"^^xsd:date ;
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/aacr> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "BiblioKeia" ;
                    bf:generationDate "2024-06-10T13:02:25"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
                    rdf:value "TESTE" ] ;
            bf:status <

In [10]:
provisionActivity = request.provisionActivity[0]
provisionActivity.place.label


'Manaus (Brazil)'

In [7]:
MDASRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")
BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
IDENTIFIERS = Namespace("http://id.loc.gov/vocabulary/identifiers/")
identifier = f"{settings.base_url}/authorities/{request.adminMetadata.identifiedBy}"
resource = URIRef(identifier)

g = Graph(identifier=identifier)
g.bind('madsrdf', MDASRDF)
g.bind('identifiers', IDENTIFIERS)
g.bind('bf', BF)


In [13]:
from pysolr import Solr
solr = Solr(f'{settings.solr}/solr/authority/', timeout=10)

In [14]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":146\n  }\n}'

In [4]:
from src.function.thesaurus.solr.updateSolr import UpdateSolr


UpdateSolr(request)

In [8]:
graph = Graph()
graph.parse('http://id.loc.gov/authorities/names/no97027235.rdf')
graph.serialize('Graeber.ttl')

<Graph identifier=Nb33df10863d54bdf9a9ceeaf5d15d56d (<class 'rdflib.graph.Graph'>)>